# Spam Classifier using Naive Bayes

This dataset was provided by Frank Kane, from Sundog Education: https://sundog-education.com/machine-learning/

In [1]:
import os
import io
import numpy
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn import preprocessing

## __Step 1: Importing the Data__

In [2]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return pd.DataFrame(rows, index=index)

data = pd.DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('emails/ham', 'nospam'))

In [3]:
data.head()

,message,class
emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, emails/spam\00001.7848dde101aa985090474a91ec93fcf0 to emails/ham\02500.05b3496ce7bca306bed0805425ec8621
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  3000 non-null   object
 1   class    3000 non-null   object
dtypes: object(2)
memory usage: 70.3+ KB


## __Step 2: Splitting into Train and Test sets and Enconding the Data__

In [5]:
X_train, X_test, y_train_class, y_test_class = train_test_split(data['message'], data['class'], random_state=42)

Enconding 'spam' and 'nospam' into 1 and 0

In [6]:
# spam --> 1, nospam --> 0
le = preprocessing.LabelEncoder()
le.fit(data["class"].values)
le.classes_

array(['nospam', 'spam'], dtype=object)

In [7]:
y_train = le.transform(y_train_class)
y_test = le.transform(y_test_class)

In [8]:
print(y_train_class[:5])
print(y_train[:5])

emails/ham\02423.1dc909c6b3be62619e6679bb3fec1ab0     nospam
emails/spam\00341.99b463b92346291f5848137f4a253966      spam
emails/ham\00600.a1c1bac6e4b69ad676c35512fec98e05     nospam
emails/ham\01044.33acf4acbd1ac5f96486771baeda20c7     nospam
emails/ham\00249.5123280972d4935e630a46f92266f6c8     nospam
Name: class, dtype: object
[0 1 0 0 0]


In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2250,)
(2250,)
(750,)
(750,)


## __Step 3: Training and fitting the model__

In [10]:
vectorizer = CountVectorizer() # CountVectorizer splits up each message into its list of words
counts_train = vectorizer.fit_transform(X_train.values) # splits each message into a list of works

classifier = MultinomialNB()
targets = y_train
classifier.fit(counts_train, targets) # fitting the model

MultinomialNB()

### On the _train_ set

In [11]:
y_pred_train = classifier.predict(counts_train)

cm = confusion_matrix(y_train, y_pred_train)
precision = precision_score(y_train, y_pred_train)     # Of all emails classified as spam by our model, how many were ACTUALLY spam?
recall = recall_score(y_train, y_pred_train)           # Of all ACTUAL spam emails, what % was detected by our model?
accuracy = accuracy_score(y_train, y_pred_train)

print(f"Confusion Matrix: {cm}")
print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"Accuracy: {round(accuracy, 3)}")

Confusion Matrix: [[1888    4]
 [  57  301]]
Precision: 0.987
Recall: 0.841
Accuracy: 0.973


### On the _test_ set

In [12]:
counts_test = vectorizer.transform(X_test.values)
y_pred_test = classifier.predict(counts_test)

cm = confusion_matrix(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)     # Of all emails classified as spam by our model, how many were ACTUALLY spam?
recall = recall_score(y_test, y_pred_test)           # Of all ACTUAL spam emails, what % was detected by our model?
accuracy = accuracy_score(y_test, y_pred_test)

print(f"Confusion Matrix: {cm}")
print(f"Precision: {round(precision, 3)}")
print(f"Recall: {round(recall, 3)}")
print(f"Accuracy: {round(accuracy, 3)}")

Confusion Matrix: [[606   2]
 [ 37 105]]
Precision: 0.981
Recall: 0.739
Accuracy: 0.948


In [13]:
# In this case, having lower recall and higher precision is better!

## __Model Evaluation__
In this particular use case, having lower recall and higher precision is better. We want our model to indentify as much spam as possible, but a user usually prefers to have a spam email in their inbox (a False Negative) than having an important non-spam email going to their spam box (False Positive).

## __Trying the model by ourselves__

In [14]:
emails = ['Get your FREE MONEY now!!!', "Hi Madalena, could you please sign this document?"]
emails_counts = vectorizer.transform(emails)
predictions = classifier.predict(emails_counts)
predictions

array([1, 0])